In [1]:
# Dependencies
import datetime as dt
import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify

In [2]:
# Database setup

engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database
Base = automap_base()

# reflect  tables
Base.prepare(engine, reflect=True)

# Save classes
Measurements = Base.classes.hawaii_measurements
Stations = Base.classes.hawaii_stations

# Create session link
session = Session(engine)


In [3]:
app = Flask(__name__)

In [4]:
# Routes
# /api/v1.0/precipitation
# api details given say "/precipitation" but instructions say to query date and tobs

@app.route("/api/v1.0/temperature")
def temperature(start_date, end_date):
   #return temperature observations for given dates
    temperature =session.query(Measurements.date,Measurements.tobs) \
             .filter(Measurements.date >= start_date).filter(Measurements.date <= end_date) \
             .all()
    #Convert the query results to a Dictionary using date as the key and tobs as the value.
    temp_results = []
    for temp in temperature:
        temperature_dict = {}
        temperature_dict["date"] = temp.date
        temperature_dict["tobs"] = temp.tobs
       
        temp_results.append(temperature_dict)

    return jsonify(temp_results)

In [5]:
# /api/v1.0/stations
@app.route("/api/v1.0/stations")
def stations():
    # Return list of stations
    stations =session.query(Stations.station,Stations.name).all()

    return jsonify(stations)

In [6]:
# /api/v1.0/tobs
#Return a json list of Temperature Observations (tobs) for the previous year
@app.route("/api/v1.0/tobs")
def tobs():
    tobs = session.query(Measurements.tobs) \
             .filter(Measurements.date >= '2016-08-03').filter(Measurements.date <= '2017-08-03').all()
    
    return jsonify(tobs)

In [7]:
# /api/v1.0/<start> and /api/v1.0/<start>/<end>
@app.route("/api/v1.0/<start>") 
def start_range(start):
    start_date = session.query(Measurements.date,func.avg(Measurements.tobs),func.min(Measurements.tobs),func.max(Measurements.tobs)) \
             .filter(Measurements.date >= start)
        
    return jsonify(single_date)

In [8]:
@app.route("/api/v1.0/<start>/<end>")
def start_end(start,end):
    range = session.query(Measurements.date,func.avg(Measurements.tobs),func.min(Measurements.tobs),func.max(Measurements.tobs)) \
             .filter(Measurements.date >= start).filter(Measurements.date <= end)
    return jsonify(range)